# Prepare CNN Data

## Setup

In [1]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWgav30SmNiB2M_1ofjS8I1fXa_WZNeNCkEtrR1P6dR7LsE4KW8MmPw



Successfully saved authorization token.


In [2]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Users/robmarty/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Parameters

In [3]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 's2'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 1 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 224
elif SATELLITE == 'l8':
    KERNEL_SIZE = 167

print(KERNEL_SIZE)

224


In [4]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [6]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn.csv'))
print(survey_df.shape)
print(survey_df.ntl_group.value_counts())

(63374, 8)
1    21000
2    13811
3    12403
4     8867
0     7293
Name: ntl_group, dtype: int64


In [7]:
# Remove observations that had issues
#survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]
survey_df = survey_df[survey_df['uid'] != 'CO201000003682']
survey_df = survey_df[survey_df['uid'] != 'CO201000003683']
survey_df = survey_df[survey_df['uid'] != 'CO201000004885']
survey_df = survey_df[survey_df['uid'] != 'IA201400180058']
survey_df = survey_df[survey_df['uid'] != 'IA201400180040']
survey_df = survey_df[survey_df['uid'] != 'IA201400300032']
survey_df = survey_df[survey_df['uid'] != 'IA201400010147']
survey_df = survey_df[survey_df['uid'] != 'NM201300000001']
survey_df = survey_df[survey_df['uid'] != 'CO201000002924']
survey_df = survey_df[survey_df['uid'] != 'CO201000002524']
survey_df = survey_df[survey_df['uid'] != 'CO201000001170']
survey_df = survey_df[survey_df['uid'] != 'CO201000003554']
survey_df = survey_df[survey_df['uid'] != 'GA201200000310']
survey_df = survey_df[survey_df['uid'] != 'IA201400180104'] # maybe OK?
survey_df = survey_df[survey_df['uid'] != 'IA201400180012']
survey_df = survey_df[survey_df['uid'] != 'IA201400180011']
survey_df = survey_df[survey_df['uid'] != 'IA201400180048']
survey_df = survey_df[survey_df['uid'] != 'IA201400300002']
survey_df = survey_df[survey_df['uid'] != 'IA201400130192']
survey_df = survey_df[survey_df['uid'] != 'IA201400140845']
survey_df = survey_df[survey_df['uid'] != 'IA201400010076']

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [8]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    
print(len(tf_record_list))

704


In [9]:
# DONT SKIP ERRORS
if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.

        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            #time.sleep(6)
            
            ## Sometimes we hit a memory error; try until we don't hit that
            
            # TODO: could say: try 3 times?
            try_extract_data = True
            while try_extract_data:
                try:

                    print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                    survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                    proto_examples_all.extend(proto_examples_i)
                    
                    try_extract_data = False
                    
                except:
                    print("Error!")
                    time.sleep(10)
                    pass
            
        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Putting 66 observations into forcnn_AL_1_1_all.tfrecord
Observation: 0/66
Observation: 1/66
Observation: 2/66
Observation: 3/66
Error!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-9-35e4ef9beeda>", line 32, in <module>
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
  File "/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py", line 443, in prep_cnn_np
    dict_ee = values_ee.getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/collection.py", line 127, in getInfo
    return super(Collection, self).getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/computedobject.py", line 98, in getInfo


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-9-35e4ef9beeda>", line 32, in <module>
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
  File "/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py", line 443, in prep_cnn_np
    dict_ee = values_ee.getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/collection.py", line 127, in getInfo
    return super(Collection, self).getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/computedobject.py", line 98, in getInfo


TypeError: can only concatenate str (not "list") to str

In [ ]:
if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.
        try:

            survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
            year_i = survey_df_yeari['year'].iloc[0]

            ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
            survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

            print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

            proto_examples_all = []
            for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
                #time.sleep(7)
                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)

            ### Save data as tf record
            out_path_i = os.path.join(out_path, tfr_i)
            print(out_path_i)
            with tf.io.TFRecordWriter(out_path_i) as writer:
                for tf_example in proto_examples_all:
                    writer.write(tf_example.SerializeToString())

            print("Success \o/")

        except:
            time.sleep(15)
            pass

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Putting 66 observations into forcnn_AL_1_1_all.tfrecord
Observation: 0/66
Observation: 1/66
Observation: 2/66
Observation: 3/66
Observation: 4/66
Observation: 5/66
Observation: 6/66
Observation: 7/66
Observation: 8/66
Observation: 9/66
Observation: 10/66
Observation: 11/66
Observation: 12/66
Observation: 13/66
Observation: 14/66
Observation: 15/66
Observation: 16/66
Observation: 17/66
Observation: 18/66
Observation: 19/66
Observation: 20/66
Observation: 21/66
Observation: 22/66
Observation: 23/66
Observation: 24/66
Observation: 25/66
Observation: 26/66
Observation: 27/66
Observation: 28/66
Observation: 29/66
Observation: 30/66
Observation: 31/66
Observation: 32/66
Observation: 33/66
Observation: 34/66
Observation: 35/66
Observation: 36/66
Observation: 37/66
Observation: 38/66
Observation: 39/66
Observation: 40/66
Observation: 41/66
Observation: 42/66
Observation: 43/66
Observation: 44/66
Observation: 45/66
Observation: 46/66
Observation: 47/66
Observation: 48/66
Observation: 49/66
Obse

Observation: 21/250
Observation: 22/250
Observation: 23/250
Observation: 24/250
Observation: 25/250
Observation: 26/250
Observation: 27/250
Observation: 28/250
Observation: 29/250
Observation: 30/250
Observation: 31/250
Observation: 32/250
Observation: 33/250
Observation: 34/250
Observation: 35/250
Observation: 36/250
Observation: 37/250
Observation: 38/250
Observation: 39/250
Observation: 40/250
Observation: 41/250
Observation: 42/250
Observation: 43/250
Observation: 44/250
Observation: 45/250
Observation: 46/250
Observation: 47/250
Observation: 48/250
Observation: 49/250
Observation: 50/250
Observation: 51/250
Observation: 52/250
Observation: 53/250
Observation: 54/250
Observation: 55/250
Observation: 56/250
Observation: 57/250
Observation: 58/250
Observation: 59/250
Observation: 60/250
Observation: 61/250
Observation: 62/250
Observation: 63/250
Observation: 64/250
Observation: 65/250
Observation: 66/250
Observation: 67/250
Observation: 68/250
Observation: 69/250
Observation: 70/250


Observation: 91/250
Observation: 92/250
Observation: 93/250
Observation: 94/250
Observation: 95/250
Observation: 96/250
Observation: 97/250
Observation: 98/250
Observation: 99/250
Observation: 100/250
Observation: 101/250
Observation: 102/250
Observation: 103/250
Observation: 104/250
Observation: 105/250
Observation: 106/250
Observation: 107/250
Observation: 108/250
Observation: 109/250
Observation: 110/250
Observation: 111/250
Observation: 112/250
Observation: 113/250
Observation: 114/250
Observation: 115/250
Observation: 116/250
Observation: 117/250
Observation: 118/250
Observation: 119/250
Observation: 120/250
Observation: 121/250
Observation: 122/250
Observation: 123/250
Observation: 124/250
Observation: 125/250
Observation: 126/250
Observation: 127/250
Observation: 128/250
Observation: 129/250
Observation: 130/250
Observation: 131/250
Observation: 132/250
Observation: 133/250
Observation: 134/250
Observation: 135/250
Observation: 136/250
Observation: 137/250
Observation: 138/250
O

Observation: 223/250
Observation: 224/250
Observation: 225/250
Observation: 226/250
Observation: 227/250
Observation: 228/250
Observation: 229/250
Observation: 230/250
Observation: 231/250
Observation: 232/250
Observation: 233/250
Observation: 234/250
Observation: 235/250
Observation: 236/250
Observation: 237/250
Observation: 238/250
Observation: 239/250
Observation: 240/250
Observation: 241/250
Observation: 242/250
Observation: 243/250
Observation: 244/250
Observation: 245/250
Observation: 246/250
Observation: 247/250
Observation: 248/250
Observation: 249/250
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_s2/tfrecords/forcnn_IA_4_3_all.tfrecord
Success \o/
Putting 249 observations into forcnn_IA_4_4_all.tfrecord
Observation: 0/249
Observation: 1/249
Observation: 2/249
Observation: 3/249
Observation: 4/249
Observation: 5/249
Observation: 6/249
Observation: 7/249
Observation: 8/249
Observation: 9/249
Observation: 10/249

In [10]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

0
1
2
3
4


IndexError: positional indexers are out-of-bounds

In [ ]:
survey_df_yeari_chunki.iloc[[4]]